In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

In [16]:
INIT_LR = 1e-4
EPOCHS = 5        
BS = 32
IMG_SIZE = (160, 160) 
DATASET_DIR = r"C:\Users\Vedant Maladkar\Downloads\archive\data"
STEPS_PER_EPOCH = 100 
VALIDATION_STEPS = 25  

In [17]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BS,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BS,
    class_mode="categorical",
    subset="validation"
)


Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [19]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))

head_model = base_model.output
head_model = AveragePooling2D(pool_size=(5,5))(head_model)
head_model = Flatten()(head_model)
head_model = Dense(128, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation="softmax")(head_model)

model = Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
    layer.trainable = False

C:\Users\Vedant Maladkar\AppData\Local\Temp\ipykernel_23780\1917583200.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))


In [20]:
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [21]:
history = model.fit(
    train_gen,
    steps_per_epoch=min(STEPS_PER_EPOCH, train_gen.samples // BS),
    validation_data=val_gen,
    validation_steps=min(VALIDATION_STEPS, val_gen.samples // BS),
    epochs=EPOCHS
)

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.8656 - loss: 0.3257 - val_accuracy: 0.9613 - val_loss: 0.1134
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - accuracy: 0.9497 - loss: 0.1358 - val_accuracy: 0.9538 - val_loss: 0.1053
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 577ms/step - accuracy: 0.9631 - loss: 0.0978 - val_accuracy: 0.9688 - val_loss: 0.1086
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 519ms/step - accuracy: 0.9582 - loss: 0.1047 - val_accuracy: 0.9725 - val_loss: 0.0782
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.9672 - loss: 0.0854 - val_accuracy: 0.9613 - val_loss: 0.1117


In [22]:
model.save("mask_detector.model.h5")
print("Model saved as mask_detector.model.h5")

Model saved as mask_detector.model.h5
